# 系统障害分析
这是财富信用卡公司的核心系统上线之后的6周障害情况，希望通过对障害的分析，帮助财富信用卡公司合理的安排维护人员的情况。对于数据的分析，主要是要找到数据的变化规律，异常数据，以及数据之间的关系。信用卡的业务主要是以月为周期变化，如用户的账单日、还款日、会计日以及月末处理等；另外，信用卡持卡人在用卡过程中，产生疑问也会通过客服的方式进行咨询，这也许会导致业务人员发起调查申请，并发现系统新的问题。对维护人员来讲，每周的周一或者特殊的业务日之后，会有大量的工作投入到系统调查工作中。接下来，将会使用Python对这6周的数据进行分析，并发现相应的规律，来帮助财富信用卡公司来合理的安排运维工作。

本次分析过程中，使用的数据是从财富信用卡公司的障害管理系统中到处的Excel文件，在Python中导入Excel文件，并初步整理数据。在分析过程中，将数据可视化，通过图表的方式来展示数据，是有效的对数据进行分析的手段。在这里使用pandas来导入，并处理数据，pyecharts来展示数据。在实际进行数据分析之前，首先配置Python环境，使用pip来管理安装包，pandas的安装命令如下：
<pre><code>$ pip install pandas</code></pre>

安装pyecharts的命令如下：
<pre><code>$ pip install pyecharts pyecharts-snapshot</code></pre>
Python的环境配置完成后，就可以使用Python进行数据分析。首先，导入在分析过程中需要用到的类库，如下：

In [1]:
import pandas as pd
import numpy as np
from pyecharts import Bar

首先，从文件系统导入障害数据，并对简单查看一下数据的签5条记录，初步了解数据中都包含什么信息。

In [2]:
filename = 'bug_infor181004.xlsx'
df = pd.read_excel(filename)
df.head()

,id,No,State,Name,Issued Date,Issued Team,Error,Priority,Important,Online/Batch,Find Team,Bug Category,Bug Category SO,Charge Team,Bug Reason,Issue,Plan Closed Date,Closed date,Cross Checking
0,HLS0101131376,B1807001,Cancel,HLS0101131376,2018-07-10 12:14:10,A12,Un-Expection Value,L,0,Batch,NaN,System Error,NaN,NaN,NaN,NaN,NaT,NaT,NaN
1,HLS0101135491,B1808001,Cancel,HLS0101135491,2018-08-09 11:44:48,M_Team,Runtime Error,NaN,1,Batch,NaN,System Error,NaN,G04,01.Test Case Omit,01.Bug,NaT,NaT,NaN
2,HLS0101142405,B1808002,Closed,HLS0101142405,2018-08-25 21:01:00,Operation,Runtime Error,NaN,0,Batch,Operation,System Error,New,B26,01.Test Case Omit,07.Based Requirements,NaT,2018-08-31 23:20:16,NaN
3,HLS0101142419,B1808003,Closed,HLS0101142419,2018-08-25 21:01:00,Operation,Runtime Error,NaN,0,Batch,Operation,System Error,New,B26,01.Test Case Omit,07.Based Requirements,NaT,2018-08-31 23:20:27,NaN
4,HLS0101142420,B1808004,Closed,HLS0101142420,2018-08-25 21:31:00,Operation,Runtime Error,NaN,1,Online,Operation,System Error,New,Z,06.Should find in This Phase,03.Change,2018-09-06 23:00:00,2018-09-08 23:06:12,Y


简单查看数据后，发现State项存在Cancel数据，那么进一步分析一下，该项目总共有几个类别，来确认哪些需要分析。

In [3]:
data = df.groupby('State').size()
data

State
Cancel      69
Closed    3313
Open       569
dtype: int64

通过计算发现，Cancel的数据总共有69条，Closed的有3313条，Open的有569条。将这个数据用柱状图表现出来，对比会更加明显。

In [4]:
# 获取数据的索引值，用来作为柱状图的标签
index = data.index
name = index.name
title = index.values.tolist()
# print(title)
# 获取柱状图的数据
values = [int(i) for i in data.values]
print(values)

[69, 3313, 569]


In [5]:
# 画简单的柱状图
bar = Bar("障害状态分类", "分类比较")
# bar.use_theme('dark')
# pyecharts 遵循所有图表都先定义数据再进行展示
bar.add(name, title, values)

#方法一： 将图表渲染输出到html页面
# bar.render()    # 如果不穿参数，默认保存到当前文件夹下

# 方法二： 直接在jupyter中展示
bar

这其中Closed将会作为进行障害分析的对象，首先选取数据。

In [6]:
df = df[df['State'] == 'Closed']
df.head()

,id,No,State,Name,Issued Date,Issued Team,Error,Priority,Important,Online/Batch,Find Team,Bug Category,Bug Category SO,Charge Team,Bug Reason,Issue,Plan Closed Date,Closed date,Cross Checking
2,HLS0101142405,B1808002,Closed,HLS0101142405,2018-08-25 21:01:00,Operation,Runtime Error,NaN,0,Batch,Operation,System Error,New,B26,01.Test Case Omit,07.Based Requirements,NaT,2018-08-31 23:20:16,NaN
3,HLS0101142419,B1808003,Closed,HLS0101142419,2018-08-25 21:01:00,Operation,Runtime Error,NaN,0,Batch,Operation,System Error,New,B26,01.Test Case Omit,07.Based Requirements,NaT,2018-08-31 23:20:27,NaN
4,HLS0101142420,B1808004,Closed,HLS0101142420,2018-08-25 21:31:00,Operation,Runtime Error,NaN,1,Online,Operation,System Error,New,Z,06.Should find in This Phase,03.Change,2018-09-06 23:00:00,2018-09-08 23:06:12,Y
5,HLS0101142415,B1808005,Closed,HLS0101142415,2018-08-25 21:49:19,Z,Runtime Error,NaN,1,Batch,IT Center,System Error,New,Z,06.Should find in This Phase,01.Bug,2018-08-25 23:00:00,2018-08-26 02:22:39,Y
6,HLS0101142416,B1808006,Closed,HLS0101142416,2018-08-25 22:04:55,Z,Runtime Error,NaN,1,Batch,IT Center,System Error,New,Z,06.Should find in This Phase,01.Bug,2018-08-25 23:00:00,2018-08-26 05:00:46,Y


查看一下，总共有多少数据，结果如下：

In [7]:
df['State'].value_counts()

Closed    3313
Name: State, dtype: int64

那么，这些障害是怎么在什么系统里的呢？

In [8]:
data = df.groupby('Online/Batch').size()
data

Online/Batch
Batch     1859
Online     993
Other      461
dtype: int64

In [9]:
name = data.index.name
title = data.index.values.tolist()
values = [int(i) for i in data.values]


In [10]:
bar = Bar("Online/Batch", "分类比较")
bar.add(name, title, values)
bar

看一下各个Team负责的障害都有多少，是什么类型的。

In [11]:
data = df.groupby(['Charge Team', 'Issue']).size()
# data

所有B开头的Team都是同一个系统的，与其他的Team比较，没有太大的价值，在这里抽取所有B开头Team的数据进行分析。

In [12]:
newdf = df[(-df['Charge Team'].isin(['G04', 'I51', 'K21', 'Z']))]
newdf

,id,No,State,Name,Issued Date,Issued Team,Error,Priority,Important,Online/Batch,Find Team,Bug Category,Bug Category SO,Charge Team,Bug Reason,Issue,Plan Closed Date,Closed date,Cross Checking
2,HLS0101142405,B1808002,Closed,HLS0101142405,2018-08-25 21:01:00,Operation,Runtime Error,NaN,0,Batch,Operation,System Error,New,B26,01.Test Case Omit,07.Based Requirements,NaT,2018-08-31 23:20:16,NaN
3,HLS0101142419,B1808003,Closed,HLS0101142419,2018-08-25 21:01:00,Operation,Runtime Error,NaN,0,Batch,Operation,System Error,New,B26,01.Test Case Omit,07.Based Requirements,NaT,2018-08-31 23:20:27,NaN
31,HLS0101142519,B1808031,Closed,HLS0101142519,2018-08-26 04:00:00,B26,Un-Expection Value,NaN,1,Batch,Function,System Error,NaN,B26,01.Test Case Omit,01.Bug,2018-08-25 23:00:00,2018-09-10 10:20:20,N
34,HLS0101142528,B1808034,Closed,HLS0101142528,2018-08-26 05:00:00,Z,Perfomance Issue,NaN,0,Batch,Maintenance Ops,System Error,New,B21,06.Should find in This Phase,07.Based Requirements,2018-08-25 23:00:00,2018-08-26 06:42:01,N
35,HLS0101142534,B1808035,Closed,HLS0101142534,2018-08-26 05:14:00,Operation,Runtime Error,NaN,0,Batch,Operation,System Error,New,B19,06.Should find in This Phase,07.Based Requirements,2018-08-25 23:00:00,2018-08-26 20:08:09,N
52,HLS0101142584,B1808052,Closed,HLS0101142584,2018-08-26 08:50:00,B21,Un-Expection Value,NaN,2,Batch,Function,System Error,NaN,B21,06.Should find in This Phase,07.Based Requirements,2018-08-26 23:00:00,2018-08-27 13:33:33,N
66,HLS0101142640,B1808066,Closed,HLS0101142640,2018-08-26 10:20:00,B25,Un-Expection Value,L,1,Batch,Project Team,System Error,NaN,B25,01.Test Case Omit,01.Bug,2018-08-31 23:00:00,2018-09-02 18:50:30,N
71,HLS0101142647,B1808071,Closed,HLS0101142647,2018-08-26 08:00:00,B25,Un-Expection Value,L,1,Batch,Project Team,System Error,NaN,B25,01.Test Case Omit,01.Bug,2018-08-26 23:00:00,2018-08-27 12:12:51,N
72,HLS0101142651,B1808072,Closed,HLS0101142651,2018-08-26 10:41:00,B25,Un-Expection Value,L,1,Batch,Project Team,System Error,NaN,B25,01.Test Case Omit,01.Bug,2018-08-31 23:00:00,2018-09-04 09:50:59,N
87,HLS0101142692,B1808087,Closed,HLS0101142692,2018-08-26 11:17:37,Z,Runtime Error,NaN,0,Batch,IT Center,System Error,New,B21,06.Should find in This Phase,04.Same Bug,NaT,2018-08-26 14:17:24,N


接下来通过柱状图来对比一下，首先看一下，各种类型的障害，在不同Team间的分布：

In [13]:
data = newdf.groupby(['Charge Team', 'Issue']).size()
# data

In [14]:
bug_category = df.groupby('Issue').size().index.values.tolist()
# bug_category

In [15]:
team_list = newdf.groupby('Charge Team').size().index.values.tolist()
# team_list

In [16]:
show_datas = []
for team in team_list:
    tmp = data[team]
    values = []
    for bug_type in bug_category:
        try:
            value = tmp[bug_type]
        except:
            value = 0
        values.append(int(value))
    # print(values)
    show_datas.append((team, bug_category, values))
# print(show_datas)

In [17]:
bar = Bar("障害分布情况", "整体情况")
for (name, title, values) in show_datas:
    bar.add(name, title, values, is_stack=True)
bar

再看一下各个Team中，不同种类障害的比较：

In [18]:
data = newdf.groupby(['Issue', 'Charge Team']).size()
# data
show_datas = []
for bug_type in bug_category:
    tmp = data[bug_type]
    values = []
    for team in team_list:
        try:
            value = tmp[team]
        except:
            value = 0
        values.append(int(value))
    # print(values)
    show_datas.append((bug_type, team_list, values))
# print(show_datas)

In [19]:
bar = Bar("Team分布情况", "整体情况")
for (name, title, values) in show_datas:
    bar.add(name, title, values, is_stack=True)
bar

看一下，障害随着时间的分布状况，如下：

In [20]:
# newdf['Issued Date'] = newdf['Issued Date'].apply(lambda x : str(x)[:10])
newdf['Issued_Year'] = newdf['Issued Date'].apply(lambda x : str(x)[:4])
newdf['Issued_Month'] = newdf['Issued Date'].apply(lambda x : str(x)[5:7])
newdf['Issued_Day'] = newdf['Issued Date'].apply(lambda x : str(x)[8:10])
# newdf

/Users/wzy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/wzy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/wzy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

In [21]:
data = newdf.groupby(['Issued_Year', 'Issued_Month', 'Issued_Day']).size()
# data

In [22]:
year_list = newdf.groupby(['Issued_Year']).size().index.values.tolist()
# year_list

In [23]:
month_list = newdf.groupby(['Issued_Month']).size().index.values.tolist()
# month_list

In [24]:
day_list = newdf.groupby(['Issued_Day']).size().index.values.tolist()
# day_list

In [25]:
show_datas = []
for year in year_list:
    tmp_year = data[year]
    for month in month_list:
        tmp_month = tmp_year[month]
        label = str(year) + str(month)
        values = []
        for day in day_list:
            try:
                value = tmp_month[day]
            except:
                value = 0
            values.append(value)  
        show_datas.append((label, day_list, values))
# show_datas

In [26]:
bar = Bar("日期分布情况", "整体情况")
for (name, title, values) in show_datas:
    # bar.add(name, title, values, is_stack=True)
    bar.add(name, title, values)
bar

系统的业务有很强的周期性，在这里周期是按照周分布的，查看各州的分布情况：

In [27]:
from datetime import datetime
def get_weekday(today):
    # print(today)
    # date = datetime.strptime(today, '%Y-%m-%d %H:%M:%S')
    return today.strftime('%A')
# get_weekday('2018-10-12')

def get_week(today):
    # date = datetime.strptime(today, '%Y-%m-%d')
    return str(today.isocalendar()[0])+str(today.isocalendar()[1])
# get_week('2018-10-28')

In [28]:
newdf['Week_Day'] = newdf['Issued Date'].apply(get_weekday)
newdf['Week'] = newdf['Issued Date'].apply(get_week)

/Users/wzy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/wzy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [29]:
data = newdf.groupby(['Week', 'Week_Day']).size()
# data

In [30]:
week_list = newdf.groupby(['Week']).size().index.values.tolist()
weekday_list = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

In [31]:
show_datas = []
i = 1
for week in week_list:
    tmp_week = data[week]
    label = 'Week' + str(i)
    i += 1
    values = []
    for weekday in weekday_list:
        try:
            value = tmp_week[weekday]
        except:
            value = 0
        values.append(value)

    show_datas.append((label, weekday_list, values))
# show_datas

In [32]:
bar = Bar("星期分布情况", "整体情况")
for (name, title, values) in show_datas:
    # bar.add(name, title, values, is_stack=True)
    bar.add(name, title, values)
bar

在信用卡的业务中，有几个特殊的业务日，如账单日，还款日，以及会计日等。在这几个业务日的当天，关注系统的运行是否正常，在业务日的翌日，客户的咨询可能会变多，因此，在这里也关注一下业务日的翌日的情况。首先，对数据整理将这几个特殊的日子标记出来。在这里的特殊日子分别是，账单日每月10号，翌日为11号；会计日为每月3号，翌日为4号；还有一个月末日，就是每月的最后一天，它的翌日是次月1号；

In [33]:
import datetime
def specialday(today):
    isleapyear = False
    # date = datetime.strptime(today, '%Y-%m-%d')
    year = int(today.strftime('%Y'))
    if (year % 4) == 0:
       if (year % 100) == 0:
           if (year % 400) == 0:
               isleapyear = True   # 整百年能被400整除的是闰年)
       else:
           isleapyear = True       # 非整百年能被4整除的为闰年

    if isleapyear:
        last_month_day = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    else:
        last_month_day = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    
    day = today.strftime('%d')
    month = today.strftime('%m')
    last_day = last_month_day[int(month) - 1]
    if int(day) == last_day:
        return (str(year) + month, 'DayL')
    elif day == '03':
        return (str(year) + month, 'Day03')
    elif day == '04':
        return (str(year) + month, 'Day04')
    elif day == '10':
        return (str(year) + month, 'Day10')
    elif day == '11':
        return (str(year) + month, 'Day11')
    elif day == '01':
        tmp = today + datetime.timedelta(days=-1)
        return (str(tmp.strftime('%Y')) + tmp.strftime('%m'), 'Day01')
    else:
        return ('', '')
    
# date = datetime.datetime.strptime('2018-08-31', '%Y-%m-%d')
# tmp = specialday(date)
# print(tmp[1])

In [34]:
newdf['Special_YearMonth'] = newdf['Issued Date'].apply(lambda x: specialday(x)[0])
newdf['Special_Day'] = newdf['Issued Date'].apply(lambda x: specialday(x)[1])
# newdf

/Users/wzy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/wzy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [35]:
spec_days = newdf[-(newdf['Special_Day'] == '')]
# spec_days

In [36]:
data = spec_days.groupby(['Special_YearMonth', 'Special_Day']).size()
data.sort_index(ascending=True, inplace=True)
data

Special_YearMonth  Special_Day
201808             Day01          41
                   DayL           24
201809             Day01          29
                   Day03          25
                   Day04          60
                   Day10          26
                   Day11          39
                   DayL           26
201810             Day03           1
                   Day04           1
dtype: int64

In [37]:
specialdaylist = ['Day03', 'Day04', 'Day10', 'Day11', 'DayL', 'Day01']
show_datas = []
yearmonthlist = spec_days.groupby(['Special_YearMonth']).size().index.values.tolist()
yearmonthlist
for month in yearmonthlist:
    tmp = data[month]
    values = []
    for day in specialdaylist:
        try:
            value = tmp[day]
        except:
            value = 0
        values.append(value)
    # print(values)
    show_datas.append((month, specialdaylist, values))

In [38]:
bar = Bar("特殊日期障害分布", "整体情况")
for (name, title, values) in show_datas:
    # bar.add(name, title, values, is_stack=True)
    bar.add(name, title, values)
bar

不同的障害类型，是什么情况呢？使用雷达图来显示一下，看看各个分布什么状况：

In [39]:
data = newdf.groupby(['Issue']).size()
# print(data)
names = data.index.values.tolist()
values = [int(i) for i in data.values.tolist()]
schema = []
for name, value in zip(names, values):
    schema.append((name, value + 10))


In [40]:
from pyecharts import Radar
radar = Radar('障害种类分布')
radar.config(schema)
radar.add("实际障害", [values], is_splitline=True, is_axisline_show=True)
radar